# FiniteHilbertTransform example

This example uses the FiniteHilbertTransform.jl package to compute the location of modes in a homogeneous plasma.

In [17]:
Pkg.add("Quadmath")



   Resolving package versions...
   Installed Quadmath ─ v0.5.9
    Updating `~/Code/FiniteHilbertTransform/Project.toml`
  [be4d8f0f] + Quadmath v0.5.9
    Updating `~/Code/FiniteHilbertTransform/Manifest.toml`
  [be4d8f0f] + Quadmath v0.5.9
Precompiling project...
  ✓ Quadmath
  ✗ FiniteHilbertTransform
  1 dependency successfully precompiled in 4 seconds (164 already precompiled, 2 skipped during auto due to previous errors)
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package


In [18]:
Pkg.add("JLLWrappers")
Pkg.add("HDF5")


   Resolving package versions...
  No Changes to `~/Code/FiniteHilbertTransform/Project.toml`
  No Changes to `~/Code/FiniteHilbertTransform/Manifest.toml`
Precompiling project...
  ✗ FiniteHilbertTransform
  0 dependencies successfully precompiled in 3 seconds (165 already precompiled, 2 skipped during auto due to previous errors)
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package
   Resolving package versions...
  No Changes to `~/Code/FiniteHilbertTransform/Project.toml`
  No Changes to `~/Code/FiniteHilbertTransform/Manifest.toml`
Precompiling project...
  ✗ FiniteHilbertTransform
  0 dependencies successfully precompiled in 3 seconds (165 already precompiled, 2 skipped during auto due to previous errors)
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package


In [10]:
using HDF5

In [ ]:
using Pkg
Pkg.add("Plots")



In [11]:

using FiniteHilbertTransform

In [19]:
import Pkg; Pkg.precompile()

Precompiling project...
  ✗ HDF5_jll
  ✗ HDF5
  ✗ FiniteHilbertTransform
  0 dependencies successfully precompiled in 6 seconds (165 already precompiled)



LoadError: The following 2 direct dependencies failed to precompile:

HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f]

Failed to precompile HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f] to /Users/mpetersen/.julia/compiled/v1.7/HDF5/jl_Sd8SRl.
[91m[1mERROR: [22m[39mLoadError: InitError: could not load library "/Users/mpetersen/.julia/artifacts/1be7004ebc29a11b63d2ad731deb3965530d6587/lib/libmpifort.12.dylib"
dlopen(/Users/mpetersen/.julia/artifacts/1be7004ebc29a11b63d2ad731deb3965530d6587/lib/libmpifort.12.dylib, 0x0001): Library not loaded: @rpath/libquadmath.0.dylib
  Referenced from: /Users/mpetersen/.julia/artifacts/1be7004ebc29a11b63d2ad731deb3965530d6587/lib/libmpifort.12.dylib
  Reason: tried: '/Users/mpetersen/.julia/artifacts/1be7004ebc29a11b63d2ad731deb3965530d6587/lib/./libquadmath.0.dylib' (no such file), '/Users/mpetersen/.julia/artifacts/1be7004ebc29a11b63d2ad731deb3965530d6587/lib/./libquadmath.0.dylib' (no such file), '/Applications/Julia-1.7.app/Contents/Resources/julia/l

Now set the parameters. The defaults will produce the image in the example folder.

In [12]:

Ku  = 200
qself = 0.5
xmax = 20.0
nOmega   = 800
Omegamin = -4.0
Omegamax = 4.0
nEta= 400
Etamin = -3.0
Etamax = -0.01


-0.01

In [13]:

"""
    get_tabomega(tabOmega::Vector{Float64}, tabEta::Vector{Float64})

Constructs the table of omega values (complex frequency) from the specified real and imaginary components.

# Arguments
- `tabOmega::Vector{Float64}`: Vector containing the real components of frequency values.
- `tabEta::Vector{Float64}`: Vector containing the imaginary components of frequency values.

# Returns
- `tabomega::Vector{Complex{Float64}}`: Vector of complex frequency values.

"""
function get_tabomega(tabOmega::Vector{Float64},tabEta::Vector{Float64})

    nOmega = size(tabOmega,1)
    nEta   = size(tabEta,1)
    nomega = nOmega*nEta

    tabomega = zeros(Complex{Float64},nomega)
    
    icount = 1 

    for iOmega=1:nOmega # Loop over the real part of the frequency
        for iEta=1:nEta # Loop over the complex part of the frequency
            tabomega[icount] = tabOmega[iOmega] + im*tabEta[iEta] # Fill the current value of the complex frequency
            icount += 1 # Update the counter
        end
    end

    return tabomega
end




get_tabomega

In [14]:
# set up the array of frequencies
tabOmega = collect(range(Omegamin,Omegamax,length=nOmega))
tabEta = collect(range(Etamin,Etamax,length=nEta))
nomega = nOmega*nEta # Total number of complex frequencies for which the dispersion function is computed.

# (flat) array of omega values to check
tabomega = get_tabomega(tabOmega,tabEta)


taba,struct_tabLeg = setup_legendre_integration(Ku,qself,xmax,PARALLEL)
@time tabIminusXi  = ComputeIminusXi(tabomega,taba,xmax,struct_tabLeg)



LoadError: UndefVarError: setup_legendre_integration not defined

In [ ]:
epsilon_real = reshape(real.(tabIminusXi),parsed_args["nEta"],parsed_args["nOmega"])
epsilon_imag = reshape(imag.(tabIminusXi),parsed_args["nEta"],parsed_args["nOmega"])
epsilon = abs.(epsilon_real .+ im * epsilon_imag)


# Plot
contour(tabOmega,tabEta,log10.(epsilon), levels=10, color=:black, #levels=[-2.0, -1.5, -1.0, -0.5, -0.25, 0.0], 
        xlabel="Re[ω]", ylabel="Im[ω]", xlims=(-4, 4), ylims=(-3, 0),
        clims=(-2, 0), aspect_ratio=:equal, legend=false)


